# **도서 추천 모델**

- 목적: 도서 기본 정보만 제공되었을 때에 사용자의 도서 선택을 현재 선택된 도서의 내용을 기반으로 연관된 도서를 추천하기 위함
- 방법: 도서 소개글을 tf-idf로 벡터화 한 데이터로 cosine 유사도를 계산하여 유사도가 높은 순서로 도서를 추천함
- 활용 데이터: 국립 중앙 도서관 - 도서별 상세정보 (2021.12 master file)

- 목차
    1. EDA&Data Cleaning
    2. TEXT 전처리
    3. 벡터화 및 유사도 계산
    4. 도서 추천 모델 생성 및 확인

In [1]:
#import libraries
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

Mounted at /content/drive


# 1. EDA & DATA Cleaning

In [2]:
#download konlpy
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 42.9 MB/s eta 0:00:00


In [3]:
#download libraries
import pandas as pd
import numpy as np

import re
from konlpy.tag import Okt
from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
#load dataset
df = pd.read_csv('/content/drive/MyDrive/NL_BO_SPECIES_MASTER_NEW_202112.csv')
df

<ipython-input-4-f1d5d9f5237f>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/NL_BO_SPECIES_MASTER_NEW_202112.csv')


,SEQ_NO,ISBN_THIRTEEN_NO,VLM_NM,TITLE_NM,AUTHR_NM,PUBLISHER_NM,PBLICTE_DE,ADTION_SMBL_NM,PRC_VALUE,IMAGE_URL,BOOK_INTRCN_CN,KDC_NM,TITLE_SBST_NM,AUTHR_SBST_NM,TWO_PBLICTE_DE,INTNT_BOOKST_BOOK_EXST_AT,PORTAL_SITE_BOOK_EXST_AT,ISBN_NO
0,6352228,9791156759270,NaN,너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이,이용신 (지은이),푸른숲,NaN,NaN,16000.0,https://image.aladin.co.kr/product/28415/8/cov...,2004년 방영한 애니메이션 &lt;달빛천사&gt;에서 주인공 루나(풀문) 역을 맡...,NaN,NaN,NaN,2021-12-03,NaN,NaN,NaN
1,6352229,9791168120877,NaN,일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이,차도하 (지은이),위즈덤하우스,NaN,NaN,15800.0,https://image.aladin.co.kr/product/28414/66/co...,“그러니 나는 말하고 싶은 것을 말하겠다”「침착하게 사랑하기」 차도하 시인 첫 에세...,NaN,NaN,NaN,2021-12-06,NaN,NaN,NaN
2,6352230,9791168120839,NaN,본격 한중일 세계사 12 - 임오군란과 통킹 위기,굽시니스트 (지은이),위즈덤하우스,NaN,NaN,14800.0,https://image.aladin.co.kr/product/28414/47/co...,한중일 관계의 결정적 분기점인 임오군란의 막전 막후를 다룬다. 러시아의 세력 확장을...,NaN,NaN,NaN,2021-12-01,NaN,NaN,NaN
3,6352231,9791168120846,NaN,즉시 기분을 바꿔드립니다 - 신기하게 마음이 편해지는 응급 처방전,"올리비아 레메스 (지은이), 김잔디 (옮긴이)",위즈덤하우스,NaN,NaN,14000.0,https://image.aladin.co.kr/product/28414/30/co...,"누구에게나 기분 구급상자가 필요하다. 하나씩 하나씩 차근차근, 좋은 기분을 쌓고 건...",NaN,NaN,NaN,2021-12-01,NaN,NaN,NaN
4,6352232,9791168120747,NaN,"오늘도 리추얼 : 음악, 나에게 선물하는 시간",정혜윤 (지은이),위즈덤하우스,NaN,NaN,15000.0,https://image.aladin.co.kr/product/28413/80/co...,나다운 일상을 만드는 사람들의 이야기를 담은 『오늘도 리추얼』 시리즈가 위즈덤하우스...,NaN,NaN,NaN,2021-12-01,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157113,6372497,9781191496148,NaN,설민석의 세계사 대모험 11 (영국 산업 혁명 편: 혁명에 맞선 자들),김정욱|설민석,단꿈아이,NaN,NaN,NaN,NaN,NaN,NaN,설민석의세계사대모험11영국산업혁명편혁명에맞선자들,김정욱설민석,NaN,Y,Y,NaN
157114,6372498,9781161755015,NaN,A Clinical Atlas: Variations of the Bones of t...,Thomas Dwight (지은이),Kessinger Publishing,NaN,NaN,NaN,https://image.aladin.co.kr/product/7275/80/cov...,NaN,NaN,aclinicalatlasvariationsofthebonesofthehandsan...,thomasdwight지은이,2010-05-23,Y,Y,NaN
157115,6372499,9781197192907,NaN,"과탑이 알려 주는 대학 생활 한 권으로 끝내기 (어서와, 대학은 처음이지?)",조유림,든든한서재,NaN,NaN,NaN,NaN,NaN,NaN,"과탑이알려주는대학생활한권으로끝내기어서와,대학은처음이지",조유림,NaN,Y,Y,NaN
157116,6372500,9781188005384,NaN,중학영문법 총정리 한권으로 끝내기 (EBS강사 혼공쌤 허준석의 &quot;쉽고 재미...,허준석|정다운,쏠티북스,NaN,NaN,NaN,NaN,NaN,NaN,중학영문법총정리한권으로끝내기ebs강사혼공쌤허준석의quot쉽고재미있게끝내는중학영문법quot,허준석정다운,NaN,Y,Y,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157118 entries, 0 to 157117
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   SEQ_NO                     157118 non-null  int64  
 1   ISBN_THIRTEEN_NO           157118 non-null  int64  
 2   VLM_NM                     39708 non-null   object 
 3   TITLE_NM                   157118 non-null  object 
 4   AUTHR_NM                   157039 non-null  object 
 5   PUBLISHER_NM               151577 non-null  object 
 6   PBLICTE_DE                 0 non-null       float64
 7   ADTION_SMBL_NM             129589 non-null  object 
 8   PRC_VALUE                  111394 non-null  float64
 9   IMAGE_URL                  98337 non-null   object 
 10  BOOK_INTRCN_CN             84205 non-null   object 
 11  KDC_NM                     143057 non-null  float64
 12  TITLE_SBST_NM              148739 non-null  object 
 13  AUTHR_SBST_NM              14

In [6]:
#필요한 열(ISBN, TITLE, AUTHOR, PUBLISHER, BOOK INTRODUCTION)만 남기기
df = df[["ISBN_THIRTEEN_NO", "TITLE_NM", "AUTHR_NM", "PUBLISHER_NM", "BOOK_INTRCN_CN"]]
df

,ISBN_THIRTEEN_NO,TITLE_NM,AUTHR_NM,PUBLISHER_NM,BOOK_INTRCN_CN
0,9791156759270,너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이,이용신 (지은이),푸른숲,2004년 방영한 애니메이션 &lt;달빛천사&gt;에서 주인공 루나(풀문) 역을 맡...
1,9791168120877,일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이,차도하 (지은이),위즈덤하우스,“그러니 나는 말하고 싶은 것을 말하겠다”「침착하게 사랑하기」 차도하 시인 첫 에세...
2,9791168120839,본격 한중일 세계사 12 - 임오군란과 통킹 위기,굽시니스트 (지은이),위즈덤하우스,한중일 관계의 결정적 분기점인 임오군란의 막전 막후를 다룬다. 러시아의 세력 확장을...
3,9791168120846,즉시 기분을 바꿔드립니다 - 신기하게 마음이 편해지는 응급 처방전,"올리비아 레메스 (지은이), 김잔디 (옮긴이)",위즈덤하우스,"누구에게나 기분 구급상자가 필요하다. 하나씩 하나씩 차근차근, 좋은 기분을 쌓고 건..."
4,9791168120747,"오늘도 리추얼 : 음악, 나에게 선물하는 시간",정혜윤 (지은이),위즈덤하우스,나다운 일상을 만드는 사람들의 이야기를 담은 『오늘도 리추얼』 시리즈가 위즈덤하우스...
...,...,...,...,...,...
157113,9781191496148,설민석의 세계사 대모험 11 (영국 산업 혁명 편: 혁명에 맞선 자들),김정욱|설민석,단꿈아이,NaN
157114,9781161755015,A Clinical Atlas: Variations of the Bones of t...,Thomas Dwight (지은이),Kessinger Publishing,NaN
157115,9781197192907,"과탑이 알려 주는 대학 생활 한 권으로 끝내기 (어서와, 대학은 처음이지?)",조유림,든든한서재,NaN
157116,9781188005384,중학영문법 총정리 한권으로 끝내기 (EBS강사 혼공쌤 허준석의 &quot;쉽고 재미...,허준석|정다운,쏠티북스,NaN


In [7]:
#check null
df.isnull().sum()

ISBN_THIRTEEN_NO        0
TITLE_NM                0
AUTHR_NM               79
PUBLISHER_NM         5541
BOOK_INTRCN_CN      72913
dtype: int64

**content-based 도서 추천 모델을 위해 소개글이 없는 행은 삭제하기로 한다.**

## Data Cleaning

In [8]:
#책 소개글 NULL 값 가진 행 모두 삭제
df.dropna(subset=["BOOK_INTRCN_CN"], inplace=True)
df

<ipython-input-8-dfe449929913>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=["BOOK_INTRCN_CN"], inplace=True)


,ISBN_THIRTEEN_NO,TITLE_NM,AUTHR_NM,PUBLISHER_NM,BOOK_INTRCN_CN
0,9791156759270,너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이,이용신 (지은이),푸른숲,2004년 방영한 애니메이션 &lt;달빛천사&gt;에서 주인공 루나(풀문) 역을 맡...
1,9791168120877,일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이,차도하 (지은이),위즈덤하우스,“그러니 나는 말하고 싶은 것을 말하겠다”「침착하게 사랑하기」 차도하 시인 첫 에세...
2,9791168120839,본격 한중일 세계사 12 - 임오군란과 통킹 위기,굽시니스트 (지은이),위즈덤하우스,한중일 관계의 결정적 분기점인 임오군란의 막전 막후를 다룬다. 러시아의 세력 확장을...
3,9791168120846,즉시 기분을 바꿔드립니다 - 신기하게 마음이 편해지는 응급 처방전,"올리비아 레메스 (지은이), 김잔디 (옮긴이)",위즈덤하우스,"누구에게나 기분 구급상자가 필요하다. 하나씩 하나씩 차근차근, 좋은 기분을 쌓고 건..."
4,9791168120747,"오늘도 리추얼 : 음악, 나에게 선물하는 시간",정혜윤 (지은이),위즈덤하우스,나다운 일상을 만드는 사람들의 이야기를 담은 『오늘도 리추얼』 시리즈가 위즈덤하우스...
...,...,...,...,...,...
157094,9791197569708,우리 없는 세상,"로빈 스티븐슨 (지은이), 이은주 (옮긴이), 황상민 (감수)",잔잔씨,작가는 사회복지사와 상담가로 오래 일했다. 그는 자살로 누군가를 잃었거나 스스로 자...
157095,9791138804486,위대한 영혼의 미소,이한옥 (지은이),좋은땅,재외 작가의 장편소설. 인고와 눈물로 점철된 코리안 뉴요커들의 이민 역정을 다룬 이...
157096,9791138804523,"잊지 말자, 나는 기적이라는 걸 - 늘 그랬던 것처럼, 신발 끈을 고쳐 맬 우리",정매일 (지은이),좋은땅,갑작스레 세상에 나 혼자 남겨진 느낌이 들 때가 있다. 주위가 암전되고 덩그러니 남...
157097,9791197549335,전념 - 나와 세상을 바꾸는 힘에 관하여,"피트 데이비스 (지은이), 신유희 (옮긴이)",상상스퀘어,오랫동안 하나의 목적을 이루기 위해 전념한 다양한 영웅들의 사례도 제시하고 있다. ...


In [9]:
#check null
df.isnull().sum()

ISBN_THIRTEEN_NO    0
TITLE_NM            0
AUTHR_NM            0
PUBLISHER_NM        0
BOOK_INTRCN_CN      0
dtype: int64

In [10]:
#평균 책 소개글 길이 확인
"""
모든 데이터셋 유사도 구하는 과정에서 out of memory 이슈로 커널이 종료되는 문제 발생하여, 데이터 셋 크기 감소
"""
average_document_length = df['BOOK_INTRCN_CN'].apply(lambda x: len(x)).mean()
longer_than_average = df[df['BOOK_INTRCN_CN'].apply(lambda x: len(x)) > average_document_length]

print("평균 책 소개글 길이 :", average_document_length)
print("평균보다 작은 길이의 소개글을 갖는 행의 개수:", len(longer_than_average))

평균 책 소개글 길이 : 137.47213348375988
평균보다 작은 길이의 소개글을 갖는 행의 개수: 45005


In [11]:
# 평균 길이보다 짧은 값을 가진 행 필터링
shorter_than_average = df[df['BOOK_INTRCN_CN'].apply(lambda x: len(x)) < average_document_length]

# 필터링된 행 삭제
df = df.drop(shorter_than_average.index)

print(f"평균보다 긴 값을 가진 행 개수: {len(df)}")

평균보다 긴 값을 가진 행 개수: 45005


In [12]:
#상위 10명의 작가
df["AUTHR_NM"].value_counts()[:10]

AUTHR_NM
지은이: 자카예프                  132
고행만 저                      117
와룡강 지음                     108
영진정보연구소 저                   93
Joji Morikawa 저 ;김시내 번역     78
글: 예종화,김영이                  78
사마달 지음                      74
동아출판 [편]                    67
지은이: 능률영어교육연구소              61
증권연수원 편저                    56
Name: count, dtype: int64

In [13]:
#상위 10개 출판사
df["PUBLISHER_NM"].value_counts()[:10]

PUBLISHER_NM
학산문화사               1097
대원씨아이                945
디앤씨미디어               560
Rok Media(로크미디어)     471
시대고시기획               438
부크크                  411
청어람                  346
영상출판미디어              337
로크미디어                314
한국학술정보               308
Name: count, dtype: int64

In [14]:
#ISBN 기준 중복 값 확인
df.loc[df.duplicated(subset=["ISBN_THIRTEEN_NO"])]

,ISBN_THIRTEEN_NO,TITLE_NM,AUTHR_NM,PUBLISHER_NM,BOOK_INTRCN_CN


In [15]:
#소개글 기준 중복값 확인
df.loc[df.duplicated(subset=["BOOK_INTRCN_CN"], keep=False)].sort_values(by=["BOOK_INTRCN_CN"])

,ISBN_THIRTEEN_NO,TITLE_NM,AUTHR_NM,PUBLISHER_NM,BOOK_INTRCN_CN
35802,9788996388418,All that dance sport,"지은이: 김명현,박형하",명현각,댄스스포츠 전문 도서인 &quot;All that Dance Sport&quot...
35803,9788996388425,All that dance sport,"지은이: 김명현,박형하",명현각,댄스스포츠 전문 도서인 &quot;All that Dance Sport&quot...
16392,9788980032679,생물II,조태산;손병옥 [공]저,대성교육출판,# 이 책의 구성과 활용법▶ 교과서 테마 특강 및 문제 풀이- 교과서 핵심 내용만을...
16391,9788980032631,공통과학:생물,조태산;손병옥 [공]저,대성교육출판,# 이 책의 구성과 활용법▶ 교과서 테마 특강 및 문제 풀이- 교과서 핵심 내용만을...
127357,9791135493423,다시 한번 아이돌 =틴타 현대 판타지 장편소설.Once again idol,지은이: 틴타,Rok Media(로크미디어),#NO환승 #NO휴덕 #저세상주접킹양산소울 가득 B급 감성부터 소름 돋는 대형 군무...
...,...,...,...,...,...
64175,9788958944836,피의 황제=캣츠아이 퓨전 판타지 장편소설.Blood emperor,캣츠아이,마루 출판사,［표지글］당신이 나라를 세울 수 있는 여의주의 힘을 가지게 된다면? 당신은 덕으로 ...
64172,9788958945628,피의 황제=캣츠아이 퓨전 판타지 장편소설.Blood emperor,캣츠아이,마루 출판사,［표지글］당신이 나라를 세울 수 있는 여의주의 힘을 가지게 된다면? 당신은 덕으로 ...
64170,9788958945307,피의 황제=캣츠아이 퓨전 판타지 장편소설.Blood emperor,캣츠아이,마루 출판사,［표지글］당신이 나라를 세울 수 있는 여의주의 힘을 가지게 된다면? 당신은 덕으로 ...
64174,9788958945048,피의 황제=캣츠아이 퓨전 판타지 장편소설.Blood emperor,캣츠아이,마루 출판사,［표지글］당신이 나라를 세울 수 있는 여의주의 힘을 가지게 된다면? 당신은 덕으로 ...


In [16]:
#중복값 삭제
df.drop_duplicates(subset=["BOOK_INTRCN_CN"], inplace=True)
df.loc[df.duplicated(subset=["BOOK_INTRCN_CN"])]

,ISBN_THIRTEEN_NO,TITLE_NM,AUTHR_NM,PUBLISHER_NM,BOOK_INTRCN_CN


In [17]:
#작가 표시방법 변경
df["AUTHR"] = df["AUTHR_NM"].str.split("(").str[0]
df

,ISBN_THIRTEEN_NO,TITLE_NM,AUTHR_NM,PUBLISHER_NM,BOOK_INTRCN_CN,AUTHR
0,9791156759270,너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이,이용신 (지은이),푸른숲,2004년 방영한 애니메이션 &lt;달빛천사&gt;에서 주인공 루나(풀문) 역을 맡...,이용신
1,9791168120877,일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이,차도하 (지은이),위즈덤하우스,“그러니 나는 말하고 싶은 것을 말하겠다”「침착하게 사랑하기」 차도하 시인 첫 에세...,차도하
2,9791168120839,본격 한중일 세계사 12 - 임오군란과 통킹 위기,굽시니스트 (지은이),위즈덤하우스,한중일 관계의 결정적 분기점인 임오군란의 막전 막후를 다룬다. 러시아의 세력 확장을...,굽시니스트
4,9791168120747,"오늘도 리추얼 : 음악, 나에게 선물하는 시간",정혜윤 (지은이),위즈덤하우스,나다운 일상을 만드는 사람들의 이야기를 담은 『오늘도 리추얼』 시리즈가 위즈덤하우스...,정혜윤
5,9791168030183,지리산 1 - 김은희 대본집,김은희 (지은이),북로그컴퍼니,국립공원 최고의 레인저 서이강과 말 못 할 비밀을 가진 신입 레인저 강현조가 산에서...,김은희
...,...,...,...,...,...,...
157094,9791197569708,우리 없는 세상,"로빈 스티븐슨 (지은이), 이은주 (옮긴이), 황상민 (감수)",잔잔씨,작가는 사회복지사와 상담가로 오래 일했다. 그는 자살로 누군가를 잃었거나 스스로 자...,로빈 스티븐슨
157095,9791138804486,위대한 영혼의 미소,이한옥 (지은이),좋은땅,재외 작가의 장편소설. 인고와 눈물로 점철된 코리안 뉴요커들의 이민 역정을 다룬 이...,이한옥
157096,9791138804523,"잊지 말자, 나는 기적이라는 걸 - 늘 그랬던 것처럼, 신발 끈을 고쳐 맬 우리",정매일 (지은이),좋은땅,갑작스레 세상에 나 혼자 남겨진 느낌이 들 때가 있다. 주위가 암전되고 덩그러니 남...,정매일
157097,9791197549335,전념 - 나와 세상을 바꾸는 힘에 관하여,"피트 데이비스 (지은이), 신유희 (옮긴이)",상상스퀘어,오랫동안 하나의 목적을 이루기 위해 전념한 다양한 영웅들의 사례도 제시하고 있다. ...,피트 데이비스


In [18]:
#check
df["AUTHR"][:10]

0                   이용신 
1                   차도하 
2                 굽시니스트 
4                   정혜윤 
5                   김은희 
6                   이미예 
7                 존 엘킹턴 
8                   김단비 
10    트레이시 애링턴, 매튜 프레더릭 
12                   성현우
Name: AUTHR, dtype: object

In [19]:
top_10_authors = df["AUTHR"].value_counts().sort_values(ascending=False).index[:10]
filtered_df = df[df["AUTHR"].isin(top_10_authors)]

filtered_df

,ISBN_THIRTEEN_NO,TITLE_NM,AUTHR_NM,PUBLISHER_NM,BOOK_INTRCN_CN,AUTHR
10931,9788984564657,엽기도사:와룡강 괴기 무예소설,와룡강 지음,청솔,"무예소설. 일초무학에 그 훈한 복호권 하나 할 줄 모르는 백수건달, 그러나 강호무림...",와룡강 지음
10932,9788984564664,엽기도사:와룡강 괴기 무예소설,와룡강 지음,청솔,엽기적인 놈이 왔다! 일초무학에 그 흔한 복호권 하나 할 줄 모르는 백수건달! 그러...,와룡강 지음
11202,9788931418750,정보처리기능사:실기 C 특별대비,영진정보연구소 저,영진닷컴,변경된 수검방식에 따라 실기 시험을 대비하는 수검자를 위하여 구성한 책이다. 변경된...,영진정보연구소 저
11302,9788984564695,철면피,와룡강 지음,청솔,"신주십존(神州十尊)의 후대(後代)를 잇기 위해, 유언을 따르기 위해 열 명의 마누라...",와룡강 지음
11303,9788984564701,철면피,와룡강 지음,청솔,신주십존 중 한사람인 황금충 선자황이 죽었다. 죽으면서 그는 곽회에게 유언을 남겼다...,와룡강 지음
...,...,...,...,...,...,...
148142,9788931462531,(이기적) ITQ 환상의 콤비 :OA master,저자: 영진정보연구소,영진닷컴,"최신 출제기준을 적용한 도서로, ITQ 한글, 엑셀, 파워포인트의 시험 경향을 분석...",저자: 영진정보연구소
148246,9788931465525,(이기적) ITQ 한글 :ver.2020,저자: 영진정보연구소,영진닷컴,"2022년 1월부터 시행되는 2020버전의 최신 출제기준을 적용한 도서로, ITQ ...",저자: 영진정보연구소
148598,9791157675210,콘크리트 기능사 :필기 실기,고행만 저,건기원,"한국콘크리트학회에서 출간된 개정판 '콘크리트 표준시방서', '콘크리트 표준시방서 해...",고행만 저
148990,9788931463125,(이기적) ITQ 파워포인트 :기본서,저자: 영진정보연구소,영진닷컴,ITQ 파워포인트 시험 경향을 분석하여 수험생들이 혼자서도 학습할 수 있도록 한 완...,저자: 영진정보연구소


In [20]:
df.drop_duplicates(subset=["TITLE_NM", "AUTHR_NM", "PUBLISHER_NM"], inplace=True)

In [21]:
top_10_authors = df["AUTHR"].value_counts().sort_values(ascending=False).index[:10]
filtered_df = df[df["AUTHR"].isin(top_10_authors)]

filtered_df

,ISBN_THIRTEEN_NO,TITLE_NM,AUTHR_NM,PUBLISHER_NM,BOOK_INTRCN_CN,AUTHR
10931,9788984564657,엽기도사:와룡강 괴기 무예소설,와룡강 지음,청솔,"무예소설. 일초무학에 그 훈한 복호권 하나 할 줄 모르는 백수건달, 그러나 강호무림...",와룡강 지음
11202,9788931418750,정보처리기능사:실기 C 특별대비,영진정보연구소 저,영진닷컴,변경된 수검방식에 따라 실기 시험을 대비하는 수검자를 위하여 구성한 책이다. 변경된...,영진정보연구소 저
11302,9788984564695,철면피,와룡강 지음,청솔,"신주십존(神州十尊)의 후대(後代)를 잇기 위해, 유언을 따르기 위해 열 명의 마누라...",와룡강 지음
11544,9788931418552,워드프로세서 3급 실기 특별대비,영진정보연구소 저,영진닷컴,기출문제 분석편에서 기존에 출제되었던 제24회 A형 시험을 따라하기식으로 분석하여 ...,영진정보연구소 저
11545,9788931418538,워드프로세서 1급 실기 특별대비,영진정보연구소 저,영진닷컴,기출문제 분석편에는 기존에 출제되었던 제24회 A형 시험을 따라하기식으로 분석하여 ...,영진정보연구소 저
...,...,...,...,...,...,...
147387,9788984751743,(중고등부 학생용)새소식 공과,프리셉트성경연구원 편,프리셉트,전부서(유치부-청장년까지) 동일 본문 통합공과한국 어린이전도협회에서 출간한 새 속식...,프리셉트성경연구원 편
148142,9788931462531,(이기적) ITQ 환상의 콤비 :OA master,저자: 영진정보연구소,영진닷컴,"최신 출제기준을 적용한 도서로, ITQ 한글, 엑셀, 파워포인트의 시험 경향을 분석...",저자: 영진정보연구소
148246,9788931465525,(이기적) ITQ 한글 :ver.2020,저자: 영진정보연구소,영진닷컴,"2022년 1월부터 시행되는 2020버전의 최신 출제기준을 적용한 도서로, ITQ ...",저자: 영진정보연구소
148598,9791157675210,콘크리트 기능사 :필기 실기,고행만 저,건기원,"한국콘크리트학회에서 출간된 개정판 '콘크리트 표준시방서', '콘크리트 표준시방서 해...",고행만 저


In [22]:
df.shape

(34852, 6)

In [23]:
df.drop(columns=["AUTHR_NM"], inplace=True)
df

,ISBN_THIRTEEN_NO,TITLE_NM,PUBLISHER_NM,BOOK_INTRCN_CN,AUTHR
0,9791156759270,너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이,푸른숲,2004년 방영한 애니메이션 &lt;달빛천사&gt;에서 주인공 루나(풀문) 역을 맡...,이용신
1,9791168120877,일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이,위즈덤하우스,“그러니 나는 말하고 싶은 것을 말하겠다”「침착하게 사랑하기」 차도하 시인 첫 에세...,차도하
2,9791168120839,본격 한중일 세계사 12 - 임오군란과 통킹 위기,위즈덤하우스,한중일 관계의 결정적 분기점인 임오군란의 막전 막후를 다룬다. 러시아의 세력 확장을...,굽시니스트
4,9791168120747,"오늘도 리추얼 : 음악, 나에게 선물하는 시간",위즈덤하우스,나다운 일상을 만드는 사람들의 이야기를 담은 『오늘도 리추얼』 시리즈가 위즈덤하우스...,정혜윤
5,9791168030183,지리산 1 - 김은희 대본집,북로그컴퍼니,국립공원 최고의 레인저 서이강과 말 못 할 비밀을 가진 신입 레인저 강현조가 산에서...,김은희
...,...,...,...,...,...
157094,9791197569708,우리 없는 세상,잔잔씨,작가는 사회복지사와 상담가로 오래 일했다. 그는 자살로 누군가를 잃었거나 스스로 자...,로빈 스티븐슨
157095,9791138804486,위대한 영혼의 미소,좋은땅,재외 작가의 장편소설. 인고와 눈물로 점철된 코리안 뉴요커들의 이민 역정을 다룬 이...,이한옥
157096,9791138804523,"잊지 말자, 나는 기적이라는 걸 - 늘 그랬던 것처럼, 신발 끈을 고쳐 맬 우리",좋은땅,갑작스레 세상에 나 혼자 남겨진 느낌이 들 때가 있다. 주위가 암전되고 덩그러니 남...,정매일
157097,9791197549335,전념 - 나와 세상을 바꾸는 힘에 관하여,상상스퀘어,오랫동안 하나의 목적을 이루기 위해 전념한 다양한 영웅들의 사례도 제시하고 있다. ...,피트 데이비스


# 2. TEXT 전처리

In [24]:
#소개글 전처리 한글제외 제거
def Remove_but_KR(text):
    pattern = r'[^ㄱ-ㅎㅏ-ㅣ가-힣]\s'
    result = re.sub(pattern, '', text)

    result = re.sub(r'[&%$#@!《》()[]{}"\dA-Za-z]', '', result)

    return result


#적용
df["BOOK_INTRCN_CN"] = df["BOOK_INTRCN_CN"].apply(Remove_but_KR)
df

,ISBN_THIRTEEN_NO,TITLE_NM,PUBLISHER_NM,BOOK_INTRCN_CN,AUTHR
0,9791156759270,너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이,푸른숲,2004년 방영한 애니메이션 &lt;달빛천사&gt;에서 주인공 루나(풀문역을 맡으며...,이용신
1,9791168120877,일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이,위즈덤하우스,“그러니 나는 말하고 싶은 것을 말하겠다”「침착하게 사랑하기차도하 시인 첫 에세이새...,차도하
2,9791168120839,본격 한중일 세계사 12 - 임오군란과 통킹 위기,위즈덤하우스,한중일 관계의 결정적 분기점인 임오군란의 막전 막후를 다룬다러시아의 세력 확장을 경...,굽시니스트
4,9791168120747,"오늘도 리추얼 : 음악, 나에게 선물하는 시간",위즈덤하우스,나다운 일상을 만드는 사람들의 이야기를 담은 『오늘도 리추얼시리즈가 위즈덤하우스에서...,정혜윤
5,9791168030183,지리산 1 - 김은희 대본집,북로그컴퍼니,국립공원 최고의 레인저 서이강과 말 못 할 비밀을 가진 신입 레인저 강현조가 산에서...,김은희
...,...,...,...,...,...
157094,9791197569708,우리 없는 세상,잔잔씨,작가는 사회복지사와 상담가로 오래 일했다그는 자살로 누군가를 잃었거나 스스로 자살 ...,로빈 스티븐슨
157095,9791138804486,위대한 영혼의 미소,좋은땅,재외 작가의 장편소설인고와 눈물로 점철된 코리안 뉴요커들의 이민 역정을 다룬 이야기...,이한옥
157096,9791138804523,"잊지 말자, 나는 기적이라는 걸 - 늘 그랬던 것처럼, 신발 끈을 고쳐 맬 우리",좋은땅,갑작스레 세상에 나 혼자 남겨진 느낌이 들 때가 있다주위가 암전되고 덩그러니 남겨진...,정매일
157097,9791197549335,전념 - 나와 세상을 바꾸는 힘에 관하여,상상스퀘어,오랫동안 하나의 목적을 이루기 위해 전념한 다양한 영웅들의 사례도 제시하고 있다이를...,피트 데이비스


In [25]:
#불용어 정의 및
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','을','를','으로','자','것','에','와','한','하다']
#형태소 단위로 토큰화
okt = Okt()

In [26]:
#불용어 제거
def Remove_stopwords(text):
    if isinstance(text, list):  # 만약 입력이 리스트인 경우
        text = ' '.join(text)
    tokens = okt.morphs(text, stem=True)
    result = [word for word in tokens if not word in stopwords]
    return ' '.join(result)

#적용
df["BOOK_INTRCN_CN"] = df["BOOK_INTRCN_CN"].apply(Remove_stopwords)
df

,ISBN_THIRTEEN_NO,TITLE_NM,PUBLISHER_NM,BOOK_INTRCN_CN,AUTHR
0,9791156759270,너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이,푸른숲,2004년 방영 애니메이션 & lt ; 달빛천사 & gt ; 에서 주인공 루나 ( ...,이용신
1,9791168120877,일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이,위즈덤하우스,“ 그렇다 나 말 하고 싶다 말 ”「 침착하다 사랑 차도 하 시인 첫 에세이 새 롭...,차도하
2,9791168120839,본격 한중일 세계사 12 - 임오군란과 통킹 위기,위즈덤하우스,한중일 관계 결정 적 분기점 인 임오군란 막다 막후 다루다 러시아 세력 확장 경계 ...,굽시니스트
4,9791168120747,"오늘도 리추얼 : 음악, 나에게 선물하는 시간",위즈덤하우스,나다 일상 만들다 사람 이야기 담다 『 오늘 리추얼 시리즈 위즈 덤 하우스 에서 추...,정혜윤
5,9791168030183,지리산 1 - 김은희 대본집,북로그컴퍼니,국립공원 최고 레인저 서 이강 말 못 비밀 가지다 신입 레인저 강현 조가 산 에서 ...,김은희
...,...,...,...,...,...
157094,9791197569708,우리 없는 세상,잔잔씨,작가 사회 복지사 상담 가로 오래 일 다그 자살 로 누군가 잃다 스스로 자살 충동 ...,로빈 스티븐슨
157095,9791138804486,위대한 영혼의 미소,좋은땅,재외 작가 장편소설 인고 눈물 로 점철 되다 코리안 뉴요커 이민 역정 다루다 이야기...,이한옥
157096,9791138804523,"잊지 말자, 나는 기적이라는 걸 - 늘 그랬던 것처럼, 신발 끈을 고쳐 맬 우리",좋은땅,갑작스레 세상 나 혼자 남겨지다 느낌 들다 때 있다 주위 암 전 되다 덩 그렇다 남...,정매일
157097,9791197549335,전념 - 나와 세상을 바꾸는 힘에 관하여,상상스퀘어,오랫동안 하나 목적 이루다 위해 전념 다양하다 영웅 사례 제시 하고 있다 통해 세상...,피트 데이비스


In [27]:
#check
df["BOOK_INTRCN_CN"][:10]

0     2004년 방영 애니메이션 & lt ; 달빛천사 & gt ; 에서 주인공 루나 ( ...
1     “ 그렇다 나 말 하고 싶다 말 ”「 침착하다 사랑 차도 하 시인 첫 에세이 새 롭...
2     한중일 관계 결정 적 분기점 인 임오군란 막다 막후 다루다 러시아 세력 확장 경계 ...
4     나다 일상 만들다 사람 이야기 담다 『 오늘 리추얼 시리즈 위즈 덤 하우스 에서 추...
5     국립공원 최고 레인저 서 이강 말 못 비밀 가지다 신입 레인저 강현 조가 산 에서 ...
6     이미 예 장편소설 잠들다 입장 수 있다 독특하다 마을 그 곳 들어오다 잠들다 손님 ...
7     회복 재생 촉진 새롭다 미래 자본주의 모델 안내서 《 그린 스완 》 추다 간 돼다 ...
8     궁리 에디 션 시리즈 4 권 저자 17 권 국내 어린이 책 에서 건지다 올리다 음식...
10    나이키 BMW 드림웍스 등 세계 유수 브랜드 광고 캠페인 제작 해오다 저자 눈 보이...
12    저자 5년 간 경험 주식 투자 경험 뿐 아니다 수많다 투자자 에게 도움 되다 주식 ...
Name: BOOK_INTRCN_CN, dtype: object

# 3. 벡터화 및 유사도 계산

In [28]:
#tf-idf 적용 후 벡터화
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df["BOOK_INTRCN_CN"])
tfidf_matrix.shape

(34852, 58217)

In [29]:
#memory 사용량 줄이기 위해 데이터 타입 변경
tfidf_matrix = tfidf_matrix.astype(np.float32)

In [30]:
#코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [31]:
#도서명기준 인덱스 재구성
indices = pd.Series(df.index, index=df["TITLE_NM"])

# 4. 도서 추천 모델 생성

In [32]:
#도서 추천 함수 생성
def get_rocomended_books(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx])) #도서별 코사인 유사도 리스트화
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) #내림차순 정렬
    sim_scores = sim_scores[1:6] #상위 5개 추출
    book_indices = [i[0] for i in sim_scores] #추출된 인덱스
    return df.iloc[book_indices][["TITLE_NM", "AUTHR"]]

In [33]:
indices[:10]

TITLE_NM
너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이                        0
일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이                               1
본격 한중일 세계사 12 - 임오군란과 통킹 위기                                         2
오늘도 리추얼 : 음악, 나에게 선물하는 시간                                           4
지리산 1 - 김은희 대본집                                                     5
달러구트 꿈 백화점 (100만부 기념 합본호 기프트 에디션) - 주문하신 꿈은 매진입니다                   6
그린 스완 - 회복과 재생을 촉진하는 새로운 경제                                         7
마음이 허기질 때 어린이책에서 꺼내 먹은 것들 - 나를 채운 열일곱 가지 맛                          8
광고학교에서 배운 101가지 - 101 Things I Learned(r) in Advertising School    10
개미 5년, 세후 55억                                                      12
dtype: int64

In [34]:
get_rocomended_books('본격 한중일 세계사 12 - 임오군란과 통킹 위기')

,TITLE_NM,AUTHR
33689,두근두근 역사여행 조선 :두루두루 방방곡곡 조선 체험기,이광희 글 ;홍수진 그림
91522,Who? 흥선 대원군·명성 황후,글: 지노 ;그림: 김광일
122768,조선전기 :조선건국 - 4대 사화,"집필 위원: 김서형,서정원"
37083,조선공학개론,"김옥삼,양현수 공저"
9784,"전통, 조약, 장사 - 청 제국주의와 조선, 1850-1910",커크 W. 라슨


**역사 관련 책 추출 확인**

In [35]:
#추출된 책 소개글 확인
df.loc[df.index.isin([33689, 91522, 122768, 37083, 9784])]

,ISBN_THIRTEEN_NO,TITLE_NM,PUBLISHER_NM,BOOK_INTRCN_CN,AUTHR
9784,9788998309053,"전통, 조약, 장사 - 청 제국주의와 조선, 1850-1910",모노그래프,조청 관계 전통 적 인 조 공 체제 적절하다 운용 되다 사례 로 언급 되다 그러나 ...,커크 W. 라슨
33689,9788961886444,두근두근 역사여행 조선 :두루두루 방방곡곡 조선 체험기,중앙북스,조선 역사 문화 지리 번 둘러보다 오감 만족 체험 여행 ! 조선 여행 전문 가이드 ...,이광희 글 ;홍수진 그림
37083,9788956135113,조선공학개론,선학출판사,조선 분야 설계 분야 혹은 현장 관리 요원 배치 되다 학생 에게 기본 적 알다 내용...,"김옥삼,양현수 공저"
91522,9791156393191,Who? 흥선 대원군·명성 황후,스튜디오 다산,Wh 한국사 조선 시대 시리즈 격변 하 조선 말 조선 이끌다 두 지도자 다루다 통상...,글: 지노 ;그림: 김광일
122768,9788964721391,조선전기 :조선건국 - 4대 사화,휘슬러,& lt ; 조선 전기 & gt ; 늘다 고려 말기 혼란 시기 극복 하고 새롭다 나...,"집필 위원: 김서형,서정원"
